In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= '0.20'

from sklearn.impute import SimpleImputer
#from sklearn.pipeline import Pipelines
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

# TensorFlow ≥2.0 is required
import tensorflow_addons as tfa
import tensorflow as tf
assert tf.__version__ >= '2.0'

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import plot_model

from keras.models import Sequential
#from keras.layers.wrappers import TimeDistributed
#from keras.layers import Dense,LSTM,Conv2D, BatchNormalization,Flatten, MaxPooling2D
#from keras.layers import Conv2DTranspose,Concatenate,UpSampling2D,Cropping2D
#from keras.layers import Input, Lambda, Reshape, Dropout, Activation

from tensorflow.keras.layers import Dropout, BatchNormalization, Reshape
from tensorflow.keras.layers import Dense, Conv2D, Input, MaxPooling2D, Flatten, MaxPool2D, MaxPool3D, UpSampling2D
from tensorflow.keras.layers import Conv2DTranspose, Flatten, Reshape, Cropping2D, Embedding, BatchNormalization,ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU, Activation, Input, add, multiply
from tensorflow.keras.layers import concatenate, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Lambda
import tensorflow.keras.backend as K
from tensorflow.keras.models import load_model 
from tensorflow.python.ops import gen_nn_ops

#print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# Common imports
import os
import glob
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import dask
import datetime
import math
import pathlib
import hashlib
dask.config.set({'array.slicing.split_large_chunks': False})

# To make this notebook's output stable across runs
np.random.seed(42)

# Config matplotlib
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt


mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
# Dotenv
from dotenv import dotenv_values
# Custom utils
from utils.utils_data import *
from utils.utils_ml import *
from utils.utils_plot import *
from utils.utils_unet import *
from utils.utils_resnet import *

In [2]:
#load test data
dg_test_X = np.array(xr.open_dataarray('tmp/data/dg_test_X.nc'))
dg_test_Y = np.array(xr.open_dataarray('tmp/data/dg_test_Y.nc'))
dg_test_Y_xtrm = np.array(xr.open_dataarray('tmp/data/dg_test_Y_xtrm.nc'))


In [3]:
ds_test = xr.open_dataset('tmp/data/ds_test.nc')

In [4]:
# Define args for the U-net model
i_shape = dg_test_X.shape[1:]
o_shape = (46,56)

print(f'X shape: {i_shape}')
print(f'y shape: {o_shape}')
output_channels = 1
num_filters = 32
use_batchnorm = True
dropout = True
lr = 0.0004
#optimizer = tf.optimizers.Adam(learning_rate = lr)
EPOCHS = 50

X shape: (46, 56, 10)
y shape: (46, 56)


In [5]:
names_mod = ['unet']
for imod in names_mod:
    tmp_file = pathlib.Path(f'tmp/{imod}')

In [6]:
custom_objects = {"weighted_cross_entropy_fn": weighted_binary_cross_entropy}
with keras.utils.custom_object_scope(custom_objects):
                m = load_model(tmp_file)

2022-04-20 08:36:05.010299: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-20 08:36:05.864570: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10415 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [7]:
m.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 46, 56, 10)  0           []                               
                                ]                                                                 
                                                                                                  
 zero_padding2d_1 (ZeroPadding2  (None, 48, 64, 10)  0           ['input_2[0][0]']                
 D)                                                                                               
                                                                                                  
 conv2d_19 (Conv2D)             (None, 48, 64, 64)   5824        ['zero_padding2d_1[0][0]']       
                                                                                            

In [8]:
# make predictions
predictions = m.predict(dg_test_X)

2022-04-20 08:36:26.628108: I tensorflow/stream_executor/cuda/cuda_dnn.cc:368] Loaded cuDNN version 8201


In [9]:
dg_test_X.shape

(2192, 46, 56, 10)

In [10]:
predictions = predictions.reshape(2192,46,56)

In [11]:
ind=np.where(np.isnan(predictions[:,0,0]))

In [12]:
# Create the visualization model
#layer_names = [layer.name for layer in m.layers]
layer_outputs = [layer.output for layer in m.layers]
feature_map_model = tf.keras.models.Model(inputs=m.input,
                                          outputs=layer_outputs)

In [13]:
#get weights
train_weights = m.trainable_weights
raw_weigths = m.weights

In [14]:
# Extract activation layers
activations = [layer.output for layer in m.layers]
activations = activations[::-1]

In [15]:
# Extract layer names and outputs
#layer_names = [layer.name for layer in m.layers]
layer_names = [layer.name for layer in m.layers if 'dropout' not in layer.name]
layer_names = layer_names[::-1]

In [16]:
def get_model_params(model):
    names, activations, weights, strids = [], [], [], []

    for layer in model.layers:
        name = layer.name if layer.name != 'predictions' else 'fc_out'
        names.append(name)
        activations.append(layer.output)
        weights.append(layer.get_weights())
        if 'conv' in name: #conv 일때 stride 값 저장
            strids.append(layer.strides)#s[0])
        else:
            strids.append([])
    return names, activations, weights, strids

In [17]:
alpha=2
epsilon=1e-7

names, activations, weights, strides = get_model_params(m)
num_layers = len(names)

In [18]:
alpha = 2
beta = 1 - alpha

In [19]:
r=m.output

In [20]:
# self.relevance = self.compute_relevances()
#        self.lrp_runner = tf.keras.backend.function(inputs=[self.model.input, ], outputs=[self.relevance, ])
def backprop_fc( w, b, a, r):
    w_p = tf.maximum(w, 0.)
    b_p = tf.maximum(b, 0.)
    z_p = tf.matmul(a, w_p) + b_p + epsilon
    
    s_p = r / z_p
    c_p = tf.matmul(s_p, tf.transpose(w_p))

    w_n = tf.minimum(w, 0.)
    b_n = tf.minimum(b, 0.)
    z_n = tf.matmul(a, w_n) + b_n - epsilon
    
    #z_n_crop = crop_output(r, z_n)
    #s_n = r / z_n_crop
    #c_n = tf.matmul(s_n, tf.transpose(w_n))

    return a * c_p#(self.alpha * c_p + self.beta * c_n)

def backprop_flatten( a, r):
    shape = a.get_shape().as_list()
    shape[0] = -1
    return tf.reshape(r, shape)

def backprop_max_pool2d(a, r, ksize=(1, 2, 2, 1), strides=(1, 2, 2, 1)):
    #z = tf.nn.pool2d(a, pool_size=ksize[1:-1], strides=strides[1:-1], padding='VALID', pool_mode='max')
    z = tf.nn.max_pool(a, ksize[1:-1], strides=strides[1:-1], padding='SAME') + epsilon

    z_p = tf.maximum(z, 0.) + epsilon
    s_p = r / z_p
    c_p = gen_nn_ops.max_pool_grad_v2(a, z_p, s_p, ksize, strides, padding='SAME')

    z_n = tf.minimum(z, 0.) - epsilon
    s_n = r / z_n
    c_n = gen_nn_ops.max_pool_grad_v2(a, z_n, s_n, ksize, strides, padding='SAME')

    return a * c_p #(self.alpha * c_p + self.beta * c_n)

def backprop_conv2d( w, b, a, r, strides=(1, 1, 1, 1)):
    w_p = tf.maximum(w, 0.)
    b_p = tf.maximum(b, 0.)
    z_p = tf.nn.conv2d(a, w_p, strides=strides, padding='SAME') + b_p + epsilon
    
 
    if (r.shape[1] > z_p.shape[1]):
        r_p_crop = crop_output(z_p, r)
        s_p = r_p_crop / z_p
    else:
        z_p_crop = crop_output(r, z_p)
        s_p = r / z_p_crop
        
    #elif(r.shape[1] < z_p.shape[1]):
    #    z_p_crop = crop_output(r, z_p)
    #    s_p = r / z_p_crop
        #s_p = r / z_p
        
    
    c_p = tf.compat.v1.nn.conv2d_backprop_input(tf.shape(a), w_p, s_p, strides, padding='SAME')

    w_n = tf.minimum(w, 0.)
    b_n = tf.minimum(b, 0.)
    z_n = tf.nn.conv2d(a, w_n, strides=strides, padding='SAME') + b_n - epsilon
    
    if (r.shape[1] > z_n.shape[1]):
        r_n_crop = crop_output(z_n, r)
        s_n = r_n_crop / z_n
    else:
        z_n_crop = crop_output(r, z_n)
        s_n = r / z_n_crop

    #s_n = r / z_n
    c_n = tf.compat.v1.nn.conv2d_backprop_input(tf.shape(a), w_n, s_n, strides, padding='SAME')
    
    return a * c_p #(self.alpha * c_p + self.beta * c_n)


In [21]:
from tensorflow.python.framework.ops import disable_eager_execution 
disable_eager_execution()

In [22]:
r = m.output
for i in range(num_layers - 2, -1, -1):
    print(i)
    print(names[i+1])
    # note: max_pooling is not working, need to figure this out
    if 'dropout' in names[i + 1] or 'normalization' in names[i+1] or 'cropping' in names[i+1] or 'concatenate' in names[i+1] or 'activation' in names[i+1] or 'conv2d_transpose' in names[i+1] or 'zero_padding2d' in names[i+1]  :
        print('r')
        r = r
    elif 'dense' in names[i + 1]:
        r = backprop_fc(weights[i + 1][0], weights[i + 1][1], activations[i], r)
    elif 'flatten' in names[i + 1]:
        r = backprop_flatten(activations[i], r)
    elif 'pool' in names[i + 1]:
        r = backprop_max_pool2d(activations[i], r)
    elif 'conv2d' in names[i + 1]:
        r = backprop_conv2d(weights[i + 1][0], weights[i + 1][1], activations[i], r)
    #elif 'conv2d_transpose' in names[i + 1]:
    #    r = backprop_conv2d(weights[i + 1][0], weights[i + 1][1], activations[i], r, strides=strides[i+1])
    elif 'conv1d' in names[i + 1]:
        r = backprop_conv1d(weights[i + 1][0], weights[i + 1][1], activations[i], r, strides=strides[i+1])
  
    else:
        raise 'Layer not recognized!'
        sys.exit()


63
cropping2d_9
r
62
conv2d_37
61
activation_31
r
60
conv2d_36
59
dropout_9
r
58
activation_30
r
57
conv2d_35
56
concatenate_7
r
55
cropping2d_8
r
54
conv2d_transpose_7
r
53
activation_29
r
52
conv2d_34
51
dropout_8
r
50
activation_28
r
49
conv2d_33
48
concatenate_6
r
47
cropping2d_7
r
46
conv2d_transpose_6
r
45
activation_27
r
44
conv2d_32
43
dropout_7
r
42
activation_26
r
41
conv2d_31
40
concatenate_5
r
39
cropping2d_6
r
38
conv2d_transpose_5
r
37
activation_25
r
36
conv2d_30
35
dropout_6
r
34
activation_24
r
33
conv2d_29
32
concatenate_4
r
31
cropping2d_5
r
30
conv2d_transpose_4
r
29
batch_normalization_3
r
28
conv2d_28
27
dropout_5
r
26
batch_normalization_2
r
25
conv2d_27
24
max_pooling2d_7


TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None, 6, 8, 512), dtype=tf.float32, name=None), name='activation_23/Relu:0', description="created by layer 'activation_23'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [25]:
a=activations[i]

In [26]:
ksize=(1, 2, 2, 1)
strides=(1, 2, 2, 1)

In [27]:
z = tf.nn.max_pool(a, ksize[1:-1], strides=strides[1:-1], padding='SAME') + epsilon

In [28]:
z

<KerasTensor: shape=(None, 3, 4, 512) dtype=float32 (created by layer 'tf.__operators__.add_35')>

In [30]:
z_p = tf.maximum(z, 0.) + epsilon
s_p = r / z_p

In [35]:
c_p = gen_nn_ops.max_pool_grad_v2(a, z_p, s_p, ksize, strides, padding='SAME')

TypeError: You are passing KerasTensor(type_spec=TensorSpec(shape=(None, 6, 8, 512), dtype=tf.float32, name=None), name='activation_23/Relu:0', description="created by layer 'activation_23'"), an intermediate Keras symbolic input/output, to a TF API that does not allow registering custom dispatchers, such as `tf.cond`, `tf.function`, gradient tapes, or `tf.map_fn`. Keras Functional model construction only supports TF API calls that *do* support dispatching, such as `tf.math.add` or `tf.reshape`. Other APIs cannot be called directly on symbolic Kerasinputs/outputs. You can work around this limitation by putting the operation in a custom Keras layer `call` and calling that layer on this symbolic input/output.

In [85]:
img = dg_test_X[0,:,:,:]

In [86]:
#image = tf.expand_dims(img, axis=0)
image = img[None,:,:,:]

In [87]:
f = K.function(inputs=m.input, outputs=r)

In [90]:
r

<KerasTensor: shape=(None, 48, 64, 10) dtype=float32 (created by layer 'tf.math.multiply_75')>

In [88]:
lrp_runner = K.function(inputs=[m.input, ], outputs=[r, ])

In [89]:
lrp_runner(image)

2022-04-19 15:28:37.094069: W tensorflow/core/framework/op_kernel.cc:1745] OP_REQUIRES failed at conv_grad_input_ops.cc:96 : INVALID_ARGUMENT: Conv2DSlowBackpropInput: Size of out_backprop doesn't match computed: actual = 46, computed = 48 spatial_dim: 1 input: 48 filter: 1 output: 46 stride: 1 dilation: 1


InvalidArgumentError: Exception encountered when calling layer "tf.compat.v1.nn.conv2d_backprop_input_116" (type TFOpLambda).

Conv2DSlowBackpropInput: Size of out_backprop doesn't match computed: actual = 46, computed = 48 spatial_dim: 1 input: 48 filter: 1 output: 46 stride: 1 dilation: 1 [Op:Conv2DBackpropInput]

Call arguments received:
  • input_sizes=tf.Tensor(shape=(4,), dtype=int32)
  • filter=tf.Tensor(shape=(1, 1, 64, 1), dtype=float32)
  • out_backprop=tf.Tensor(shape=(1, 46, 56, 1), dtype=float32)
  • strides=('1', '1', '1', '1')
  • padding='SAME'
  • use_cudnn_on_gpu=True
  • data_format=NHWC
  • dilations=[1, 1, 1, 1]
  • name=None
  • filters=None

In [ ]:
def run(image):
        """Computes feature relevance scores for single image
        :param image: ndarray of shape (W, H, C)
        :return: features_relevance_scores: ndarray of same size as input image
        """
        image = preprocess_input(image)
        image = tf.expand_dims(image, axis=0)
        relevance_scores = self.f(inputs=image)
        relevance_scores = self.postprocess(relevance_scores)
        return np.squeeze(relevance_scores)

In [ ]:
def heatmap(image, cmap_type='rainbow', reduce_op='sum', reduce_axis=-1, **kwargs):
    cmap = get_cmap(cmap_type)
    shape = list(image.shape)
    reduced_image = reduce_channels(image, axis=reduce_axis, op=reduce_op)
    projected_image = project_image(reduced_image, **kwargs)
    heatmap = cmap(projected_image.flatten())[:, :3].T
    heatmap = heatmap.T
    shape[reduce_axis] = 3
    return heatmap.reshape(shape)

def get_heatmaps(gammas, cmap_type='rainbow', **kwargs):
    heatmaps = [heatmap(g, cmap_type=cmap_type, **kwargs) for g in gammas]
    return heatmaps